In [1]:
from ortools.sat.python import cp_model
import pandas as pd

In [2]:
# Read CSV
class_list = pd.read_csv("class_list.csv")
timeslots_days = pd.read_csv("timeslots.csv")

lessons_required = pd.read_csv("lessons_required.csv")
weekday = pd.read_csv("weekdays.csv")
teacher_classes = pd.read_csv("teacher_classes.csv")
student_classes = pd.read_csv("student_classes.csv")

teacher_list = pd.read_csv("teacher_list.csv")

In [3]:
student_classes = student_classes[student_classes['class']!='isizulu-12']

In [4]:
classes = list(class_list['class'])# + ['isizulu-12']
timeslots = list(timeslots_days['timeslot'])
days = list(weekday['weekday'])
# periods = range(9)
teacher = list(teacher_list['teacher'])

num_lessons = dict(zip(lessons_required["class"], lessons_required["num_lessons"]))
class_teacher = dict(zip(teacher_classes["class"], teacher_classes["teacher"]))
class_students = dict(zip(student_classes["class"], student_classes["student"]))

In [5]:
num_lessons['isizulu-10'] = 0

In [6]:
model = cp_model.CpModel()

## Variables

We’ll create one **binary decision variable** per `(class, day, period)`

In [7]:
# Variables
x = {}
for c in classes:
    for t in timeslots:
        x[(c,t)] = model.NewBoolVar(f'x_{c}_{t}')

## Constraints

### Required lessons per class

Each class must be scheduled the exact number of times specified in `num_lessons`.

In [8]:
# Constraint: each class gets required lessons
for c in classes:
    model.Add(sum(x[(c,t)] for t in timeslots) == num_lessons[c])

### Teachers cannot be double-booked

For each `(day, period)`, a teacher can only teach one of their classes.

In [9]:
for t in timeslots:
    for ct in set(class_teacher.values()):  # each teacher
        teaching_classes = [cls for cls, teacher in class_teacher.items() if teacher == ct]
        if teaching_classes:
            model.Add(sum(x[(c,t)] for c in teaching_classes if c in classes) <= 1)

### Students cannot be double-booked

Same idea: For each `(day, period)`, a student can only attend one class.

In [10]:
for t in timeslots:
    for s in set(class_students.values()):
        enrolled_classes = [c for c, students in class_students.items() if s in students]
        if enrolled_classes:
            model.Add(sum(x[(c,t)] for c in enrolled_classes if c in classes) <= 1)

### Spread lessons across days (soft preference)

For now, we’ll keep it simple, but later we can add an **objective** to penalize imbalance.

In [11]:
for c in classes:
    lessons = num_lessons[c]
    for d in days:
        if lessons <= 4:
            # Spread: at most 1 per day
            model.Add(sum(x[(c, t)] for p in timeslots) <= 1)
        else:
            # Allow 2 per day if more than 5 lessons in total
            model.Add(sum(x[(c, t)] for p in timeslots) <= 2)

## Step 3. Solver & Output

In [12]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
status = solver.Solve(model)

In [13]:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    rows = []
    for (c, t), var in x.items():
        if solver.Value(var) == 1:
            slots = f"{t}"
            rows.append([slots, c])
    output = pd.DataFrame(rows, columns=["timeslot", "class"])
    output.to_csv("timetable2.csv", index=False)
    print("Solution written to timetable_solution.csv")
else:
    print("No feasible solution found")

Solution written to timetable_solution.csv


In [14]:
# python opt_test.py timetable2.csv